In [1]:
#!pip install spacy
#!pip freeze | grep torch
#!pip install torchtext!
#download spacy library first
#!python -m spacy download en_core_web_sm

In [2]:
import spacy
import torch
import torchtext
import torchtext.data
import torch.nn as nn
import torch.nn.functional as F
from torchtext.vocab import Vocab
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

nlp = spacy.load('en_core_web_sm')


In [4]:
device = torch.device("cuda:5" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [4]:
!pip install torchtext

     |████████████████████████████████| 81kB 99kB/s eta 0:00:011
     |████████████████████████████████| 1.1MB 90kB/s eta 0:00:015


In [5]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx) 
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)

        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        #text = text.permute(1, 0)
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)

        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
                
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [6]:
#Loads pretrained model and sets the model to eval mode
model = torch.load('models/imdb-model-cnn.pt')
model.eval()
model = model.to(device)

FileNotFoundError: [Errno 2] No such file or directory: 'models/imdb-model-cnn.pt'